In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.fft import fft
from pytorch_metric_learning import losses

import numpy as np
import pandas as pd

In [2]:
# Data Augmentation Functions
def temporal_augmentation(x, alpha=0.4, beta=0.5):
    # random = torch.randint(0,1,(1,))
    # if random == 0:
    jitter = torch.normal(mean=0.0, std=alpha, size=x.shape)
    return x + jitter
    # else:
    #     scale = torch.empty(1).uniform_(1-beta,1+beta).item()
    #     return x * scale


def frequency_augmentation(x):
    # Example: Mask random frequency components
    mask = torch.rand(x.size()).to(x.device)
    mask[mask < 0.1] = 0  # Randomly zero out half of the components
    return x * mask

In [3]:
class TemporalEncoder(nn.Module):
    def __init__(self):
        super(TemporalEncoder, self).__init__()
        self.bilstm = nn.LSTM(1, 2, batch_first=True, bidirectional=True)
        # self.layer = nn.Linear(2, 1)

    def forward(self, x):
        batch_size = x.size(0)
        seq_len = x.size(1)
        x, _ = self.bilstm(x.view(batch_size, seq_len,1))
        # x = self.layer(x).view(batch_size, seq_len)
        return x

In [4]:
class FrequencyEncoder(nn.Module):
    def __init__(self, batch_size, seq_len):
        super(FrequencyEncoder, self).__init__()
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.batchnorm1 = nn.BatchNorm1d(8)
        self.batchnorm2 = nn.BatchNorm1d(16)
        self.batchnorm3 = nn.BatchNorm1d(32)
        self.pool = nn.MaxPool1d(kernel_size=3,stride=2)
        self.dropout = nn.Dropout()
        self.layer1 = nn.Linear(32*13, self.seq_len)
        self.layer2 = nn.Linear(self.seq_len, self.seq_len)

    def forward(self, x):
        x = x.real
        x = x.view(self.batch_size,1,-1)
        x = self.pool(F.relu(self.batchnorm1(self.conv1(x))))
        x = self.dropout(x)
        x = self.pool(F.relu(self.batchnorm2(self.conv2(x))))
        x = self.pool(F.relu(self.batchnorm3(self.conv3(x))))
        x = self.layer1(x.view(self.batch_size,32*13))
        x = self.layer2(x)
        return x

In [5]:
class InstanceLevelTransformation(nn.Module):
    def __init__(self, input_size, output_size):
        super(InstanceLevelTransformation, self).__init__()
        self.layer1 = nn.Linear(1000*4, 1000*2)
        self.layer2 = nn.Linear(1000*2, 1000)

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = F.normalize(x, p=2, dim=1)
        return x

In [6]:
class ClusterLevelTransformation(nn.Module):
    def __init__(self, input_size,cluster_size):
        super(ClusterLevelTransformation, self).__init__()
        self.layer1 = nn.Linear(1000*4, 1000*2)
        self.layer2 = nn.Linear(1000*2, 1000)
        self.layer3 = nn.Linear(1000, cluster_size)

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = F.softmax(x, dim=1)
        return x

In [7]:
# Model Definition
class CDCCModel(nn.Module):
    def __init__(self, seq_len=17520, feature_length=1752, cluster_size=10):
        super(CDCCModel, self).__init__()
        self.temporal_encoder = TemporalEncoder()
        self.temporal_transform = InstanceLevelTransformation(seq_len,feature_length)
        self.cluster_transform = ClusterLevelTransformation(seq_len,cluster_size)

    def forward(self, x, train=True):
        if train:
            x_t, x_t_aug = x           

            # Encodings
            h_t = self.temporal_encoder(x_t)
            h_t_aug = self.temporal_encoder(x_t_aug)

            # Transformations
            z_i = self.temporal_transform(h_t)
            z_i_aug = self.temporal_transform(h_t_aug)
            z_c = self.cluster_transform(h_t)
            z_c_aug = self.cluster_transform(h_t_aug)
            
            return z_i,z_i_aug,z_c,z_c_aug
        else:
            return torch.argmax(self.cluster_transform(self.temporal_encoder(x)),dim=1)

In [1]:
import pandas as pd


data = pd.read_csv("Final_Energy_dataset.csv")
data['Date'] = pd.to_datetime(data['Date'])
data = data[data['Date'].dt.date != pd.to_datetime('2012-02-29').date()]    
prosumption = {}
for i in range(101,301):
    if (data[f'load_{i}'].isna().sum() == 0) & (data[f'pv_{i}'].isna().sum() == 0):
        prosumption[f'prosumption_{i}'] = data[f'load_{i}'] - data[f'pv_{i}']
data_train_df = pd.DataFrame(prosumption).reset_index(drop=True)
data_train_df.to_csv('data_train.csv')

# # data_train_tensor = torch.Tensor(data_train_df.values).reshape(600,17520)
# data = data[((data['Date'].dt.month >= 7) & (data['Date'].dt.year == 2010)) 
#             | (((data['Date'].dt.month < 7) & (data['Date'].dt.year == 2011))) 
#             | (data['Date'] == pd.to_datetime('2011-07-01 00:00:00'))]
# prosumption = {}
# for i in range(1,101):
#     if (data[f'load_{i}'].isna().sum() == 0) & (data[f'pv_{i}'].isna().sum() == 0):
#         prosumption[f'prosumption_{i}'] = data[f'load_{i}'] - data[f'pv_{i}']
# data_test_df = pd.DataFrame(prosumption).reset_index(drop=True).T
# # data_test_df.to_csv('data_test.csv')
# data_test_df
# data_test_tensor = torch.Tensor(data_test_df.values).reshape(100,17520)

In [9]:
def get_batch(tensor, batch_size):
    indices = np.random.choice(tensor.shape[0], batch_size, replace=False)
    return tensor[indices,0:1000]

In [10]:
class InfocNCELoss(nn.Module):
    def __init__(self, temperature=0.07):
        super(InfocNCELoss, self).__init__()
        self.temperature = temperature

    def forward(self, original, augmented):
        # Batch size
        batch_size = original.size(0)

        # Normalize original and augmented embeddings
        original = F.normalize(original, p=2, dim=1)   # (batch_size, embedding_dim)
        augmented = F.normalize(augmented, p=2, dim=1)  # (batch_size, embedding_dim)

        # Concatenate original and augmented embeddings (2 * batch_size, embedding_dim)
        embeddings = torch.cat([original, augmented], dim=0)

        # Compute similarity matrix (2 * batch_size, 2 * batch_size)
        similarity_matrix = torch.mm(embeddings, embeddings.T)

        # Create labels: For each anchor in the batch, the positive pair is the corresponding augmented one
        labels = torch.cat([torch.arange(batch_size) + batch_size, torch.arange(batch_size)])

        # Mask out self-similarity to avoid considering the same sample as both positive and negative
        mask = torch.eye(2 * batch_size, dtype=torch.bool).to(original.device)
        similarity_matrix.masked_fill_(mask, -float('inf'))

        # Scale similarity matrix by temperature
        similarity_matrix /= self.temperature

        # Compute the InfoNCE loss
        loss = F.cross_entropy(similarity_matrix, labels, reduction='mean')

        return loss

In [11]:
batch_size = 16
cluster_size = 10
epochs = 50

model = CDCCModel(cluster_size=cluster_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
# loss_func = losses.SelfSupervisedLoss(losses.NTXentLoss(),symmetric=True)
# cross_entropy_loss = torch.nn.CrossEntropyLoss()
loss_func = InfocNCELoss()

for epoch in range(epochs):

    data = get_batch(data_train_tensor, batch_size)
    data_aug = temporal_augmentation(data)

    model.zero_grad()
    # print(model(data_test_tensor[0:10],train=False))

    
    
   
    
    a,b,c,d = model((data,data_aug))
    
    loss_i = (loss_func(a,b)+loss_func(b,a))/2
    # loss_c = (loss_func(c.T,d.T)+loss_func(d.T,c.T))/2

    # p_c = torch.mean(c, dim=0)
    # p_c_aug = torch.mean(d, dim=0)
    # p_c_clamp = torch.clamp(p_c, 1e-16, 1-1e-16)
    # p_c_aug_clamp = torch.clamp(p_c_aug, 1e-16, 1-1e-16)
    # loss_ce = -torch.sum(p_c*torch.log(p_c_clamp)-p_c_aug*torch.log(p_c_aug_clamp))
    # print(loss_i,loss_c,loss_ce)
    loss = loss_i #+ loss_c + loss_ce
    print(f'Epoch {epoch+1} / {epochs}, Loss: {loss.item()}')



    loss.backward()
    optimizer.step()
   

Epoch 1 / 50, Loss: 2.200089931488037
Epoch 2 / 50, Loss: 3.3343005180358887
Epoch 3 / 50, Loss: 3.1278891563415527
Epoch 4 / 50, Loss: 2.98081374168396
Epoch 5 / 50, Loss: 2.863985776901245
Epoch 6 / 50, Loss: 2.6335625648498535
Epoch 7 / 50, Loss: 2.3727290630340576
Epoch 8 / 50, Loss: 2.2593278884887695
Epoch 9 / 50, Loss: 1.8169331550598145
Epoch 10 / 50, Loss: 1.919558048248291
Epoch 11 / 50, Loss: 1.8670063018798828
Epoch 12 / 50, Loss: 2.0564041137695312
Epoch 13 / 50, Loss: 1.8606932163238525
Epoch 14 / 50, Loss: 1.699446201324463
Epoch 15 / 50, Loss: 1.3470531702041626
Epoch 16 / 50, Loss: 1.5484182834625244
Epoch 17 / 50, Loss: 1.7068978548049927
Epoch 18 / 50, Loss: 1.4273288249969482
Epoch 19 / 50, Loss: 1.3647443056106567
Epoch 20 / 50, Loss: 1.6298375129699707
Epoch 21 / 50, Loss: 1.57668936252594
Epoch 22 / 50, Loss: 1.3508992195129395
Epoch 23 / 50, Loss: 1.1075174808502197
Epoch 24 / 50, Loss: 1.230635404586792
Epoch 25 / 50, Loss: 0.8164557814598083
Epoch 26 / 50, Los

In [12]:
model(data_test_tensor[0:100,0:1000],train=False)

tensor([3, 1, 8, 6, 1, 1, 1, 6, 6, 3, 6, 1, 6, 3, 3, 6, 1, 1, 1, 3, 6, 3, 1, 1,
        1, 8, 1, 6, 1, 1, 6, 1, 8, 6, 8, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 6, 1, 6,
        1, 1, 8, 6, 6, 6, 1, 8, 6, 6, 8, 6, 1, 6, 1, 1, 1, 6, 3, 8, 3, 6, 6, 6,
        1, 6, 6, 6, 3, 8, 1, 1, 3, 6, 1, 1, 6, 6, 3, 1, 1, 1, 1, 8, 1, 6, 3, 1,
        1, 8, 1, 1])

In [29]:
import torch
from CDCC.data import CustomDataset 


cds = CustomDataset(torch.Tensor([[0,1],[2,3],[4,5]]), torch.Tensor([[6,7],[8,9],[10,11]]))
train_loader = torch.utils.data.DataLoader(dataset=cds, batch_size=1, shuffle=True)
for step, (x,y) in enumerate(train_loader):
    print(step)
    print(x,y)

0
tensor([[4., 5.]]) tensor([[10., 11.]])
1
tensor([[2., 3.]]) tensor([[8., 9.]])
2
tensor([[0., 1.]]) tensor([[6., 7.]])


In [16]:
def one_hot_encoding(X):
    X = [int(x) for x in X]
    n_values = np.max(X) + 1
    b = np.eye(n_values)[X]
    return b

In [5]:
import numpy as np

In [23]:
li = np.random.randint(0, 3, size=[10])
li_onehot = one_hot_encoding(li)


aug_1 = torch.randn((10,1,5))
aug_2 = torch.randn((10,1,5))
aug_3 = torch.randn((10,5))

In [24]:
aug_1

tensor([[[ 0.5793, -1.2249,  0.0726, -0.5853, -1.6182]],

        [[ 0.3280,  1.2782, -0.6666,  1.2716,  0.0961]],

        [[ 1.0612,  0.2259,  0.7334, -0.0340, -1.4629]],

        [[ 0.2259,  0.3952,  0.7407, -0.6767, -0.1268]],

        [[-0.2369, -0.1663, -0.6886,  0.6644, -0.2680]],

        [[ 0.9437, -0.9627, -1.3485, -1.6521,  0.1040]],

        [[ 0.1203,  2.4441,  0.3534,  2.6402, -2.1426]],

        [[ 1.4571,  0.5752, -1.3446,  0.8272, -0.2093]],

        [[ 0.5552, -1.6741,  1.4697, -0.7520,  1.0526]],

        [[ 1.3351,  0.3985,  1.6254, -0.5584, -0.0519]]])

In [25]:
li_onehot[:,0]

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1.])

In [26]:
aug_1[1-li_onehot[:, 0]] = 0
aug_1

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 1.0612,  0.2259,  0.7334, -0.0340, -1.4629]],

        [[ 0.2259,  0.3952,  0.7407, -0.6767, -0.1268]],

        [[-0.2369, -0.1663, -0.6886,  0.6644, -0.2680]],

        [[ 0.9437, -0.9627, -1.3485, -1.6521,  0.1040]],

        [[ 0.1203,  2.4441,  0.3534,  2.6402, -2.1426]],

        [[ 1.4571,  0.5752, -1.3446,  0.8272, -0.2093]],

        [[ 0.5552, -1.6741,  1.4697, -0.7520,  1.0526]],

        [[ 1.3351,  0.3985,  1.6254, -0.5584, -0.0519]]])

In [28]:
aug_2[1 - li_onehot[:, 1]] = 0
aug_2 + aug_1

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 1.6630, -0.1142,  1.4622, -1.5548, -3.4478]],

        [[ 0.1657,  1.3081, -0.5880,  0.0646,  0.0420]],

        [[-1.1388,  0.0850, -2.1036, -0.2397,  1.6391]],

        [[-0.4124, -2.2816, -1.6095, -0.5997, -0.2591]],

        [[-0.0450,  2.0129,  1.7478,  2.6780, -0.9558]],

        [[ 1.9480,  0.6407, -0.8891,  2.4291, -0.1961]],

        [[-0.4355, -2.4739,  1.8967, -1.6155, -0.1899]],

        [[ 1.2263, -0.0360,  1.3802, -0.6694,  1.5923]]])

In [63]:
aug_3[1 - li_onehot[:, 2]] = 0
aug_3

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2232, -0.2033,  0.2780,  1.5442, -0.2986],
        [ 0.5432, -0.8082,  0.8530,  0.5093,  0.9201],
        [ 0.6298,  0.3081, -0.4960, -0.6870,  1.2107],
        [ 1.3851, -0.0438,  0.9864,  1.5693, -0.0670],
        [-2.5290,  0.3443,  0.8076,  2.0789, -1.1172],
        [-0.6643, -0.4576, -2.0311,  0.4557, -1.2855],
        [-0.1285,  0.1837,  0.3927,  0.3390, -0.0385],
        [-0.5163, -2.0811,  1.6169,  0.2710,  1.4350]])

In [1]:
from CDCC.augmentations import DataTransform_T
import torch

data = torch.randn((10,1,5))
print(data)
og, aug, aug_1, aug_2 = DataTransform_T(data)

tensor([[[ 0.3416, -0.8452, -0.6422,  0.0633, -0.4780]],

        [[-0.9844,  0.5087,  0.0307, -1.6512,  0.4128]],

        [[-0.4520,  1.7438, -0.3762,  1.2024,  0.7313]],

        [[ 1.5930, -1.3147, -1.8538,  0.6295,  0.9402]],

        [[ 0.8831, -1.5312, -0.9452, -0.5693,  1.9478]],

        [[-0.1362,  0.3734, -1.1859,  0.4234, -0.5096]],

        [[-0.6835,  0.2497, -0.0903,  0.2238, -0.9135]],

        [[-0.4787,  2.3719,  2.6252,  0.9116, -0.3260]],

        [[ 0.4229, -0.8545, -0.9561,  1.2805, -0.4317]],

        [[ 0.4608,  0.2143,  0.8601,  1.2083,  0.3726]]])


In [2]:
aug_1

tensor([[[ 0.8460,  0.3896, -1.9531,  0.6685, -0.4961]],

        [[-0.4794,  1.8077,  0.5057, -1.7047,  0.3446]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.9263,  0.0098, -1.0915, -1.0077,  2.1601]],

        [[-0.2912,  0.1276, -1.2833, -0.6458, -0.3417]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[-1.0142,  2.4772,  2.5777,  0.9885, -1.2949]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0462,  2.3482,  1.1745,  1.7759,  0.8536]]], dtype=torch.float64)

In [3]:
aug_2

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.3368,  4.5153, -0.8592,  4.0130,  0.9832]],

        [[-0.0084, -2.3238,  0.3291,  2.2169,  1.3606]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[-2.6010,  0.5544, -0.2498,  0.2932, -0.1370]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.6564, -1.1039, -0.0147,  3.1929, -1.0673]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]], dtype=torch.float64)

In [4]:
aug

tensor([[[ 0.8460,  0.3896, -1.9531,  0.6685, -0.4961]],

        [[-0.4794,  1.8077,  0.5057, -1.7047,  0.3446]],

        [[-0.3368,  4.5153, -0.8592,  4.0130,  0.9832]],

        [[-0.0084, -2.3238,  0.3291,  2.2169,  1.3606]],

        [[ 0.9263,  0.0098, -1.0915, -1.0077,  2.1601]],

        [[-0.2912,  0.1276, -1.2833, -0.6458, -0.3417]],

        [[-2.6010,  0.5544, -0.2498,  0.2932, -0.1370]],

        [[-1.0142,  2.4772,  2.5777,  0.9885, -1.2949]],

        [[ 0.6564, -1.1039, -0.0147,  3.1929, -1.0673]],

        [[ 0.0462,  2.3482,  1.1745,  1.7759,  0.8536]]], dtype=torch.float64)

In [29]:
np.random.normal(loc=0., scale=0.1, size=(10,1,5))

array([[[ 0.02176858,  0.01144502, -0.08667886,  0.00933849,
         -0.05837898]],

       [[ 0.12338087, -0.06858338, -0.0408963 ,  0.08756479,
         -0.02262905]],

       [[-0.04923796, -0.06444042,  0.06061122,  0.04512797,
         -0.06934294]],

       [[-0.06387301, -0.08351301, -0.0648921 , -0.10544478,
         -0.06138439]],

       [[-0.09932564,  0.01588636, -0.06083638,  0.09049226,
          0.06108832]],

       [[ 0.11520715, -0.06738591,  0.14733264, -0.09010899,
          0.05196283]],

       [[ 0.00752993, -0.11395712,  0.15157864,  0.22666766,
          0.0009768 ]],

       [[-0.10626585, -0.17359035, -0.01501222, -0.11532472,
         -0.1030506 ]],

       [[ 0.00765463, -0.00215181,  0.1716162 , -0.24035026,
          0.06983744]],

       [[ 0.07757059, -0.03414363,  0.02834073, -0.02988733,
         -0.15754228]]])

In [5]:
import torch
import numpy as np
x = torch.randn((10,1,5))
x

tensor([[[ 0.1218,  0.0164, -0.8406,  0.4303, -0.8673]],

        [[-1.6456,  1.3475, -0.6096,  1.1897, -0.9227]],

        [[ 0.4306, -1.4497,  0.7396,  0.4622,  0.0610]],

        [[ 0.7240, -0.4813,  0.1264, -0.4526,  1.6689]],

        [[ 0.0981,  0.7305,  1.1890,  0.1520, -0.8134]],

        [[-0.8173,  0.0363, -0.6241, -1.4054,  1.5949]],

        [[ 0.5530, -0.4701,  0.2775,  0.5016, -1.0726]],

        [[-0.9671,  0.9827, -0.8305,  0.2212,  1.0194]],

        [[-0.4848,  0.6745,  0.8543, -0.8525,  0.7640]],

        [[-1.3935,  0.1515,  0.7019, -1.1955, -0.4908]]])

In [7]:
len(np.unique(x[1]))

5